# Spectral Extraction
This notebook is a sandbox for the optimal spectral extraction routine.

In [1]:
# Imports
import copy
import numpy as np
import lmfit
from astropy.modeling import models
from astropy import units as q
from awesimsoss import make_trace as mt
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
sosspsf = mt.get_SOSS_psf(1, filt='CLEAR', psfs=None, cutoff=0)

def psf_soss(scale, wavelength=1, pix=76, xy=50):
    """Make a psf of a given size and scale"""
    return copy.copy(sosspsf)*scale

def make_frame_soss(n_psfs, offset, pix, xy, plot=False, **scales):
    """Make a frame with N psfs with a pixel width and a pixel offset for each psf
    where each psf is scaled randomly"""
    # Set the values
    frame = np.zeros((76, 76+n_psfs))
    
    # Make the frame
    for n in range(n_psfs):
        frame[:, n:n+pix] += psf_soss(scales['w{}'.format(n)], pix=pix)
        
    # Trim off excess
    frame = frame[:, 38:-38]
    
    if plot:
        # Plot the frame
        fig = figure(tooltips=[('x', '$x'), ('y', '$y'), ("value", "@image")])
        data = dict(image=[frame])
        fig.image(source=data, x=0, y=0, image='image', dw=frame.shape[0], dh=frame.shape[1], palette="Inferno256")
        show(fig)

    return frame

def calc_spectrum_soss(n_psfs, offset, pix, xy, plot=False, **scales):
    """Calculate a spectrum by column sum"""
    frame = make_frame_soss(n_psfs, offset, pix, xy, plot=plot, **scales)
    spectrum = np.sum(frame, axis=0)
    
    return spectrum

def lmfitter_soss(frame, model, uncertainty=None, method='powell', verbose=True, **kwargs):
    """Use lmfit to find the scale of each psf

    Parameters
    ----------
    frame: sequence
        The frame data
    model: function
        The model to fit

    Returns
    -------
    lmfit.Model.fit.fit_report
        The results of the fit
    """
    # Make a spectrum
    spectrum = np.sum(frame, axis=0)
    x = np.arange(len(spectrum))
    
    # Initialize lmfit Params object
    initialParams = lmfit.Parameters()

    # Concatenate the lists of parameters
    param_list = [('w{}'.format(n), 1., True) for n in range(n_psfs)]
    param_list.append(('offset', offset, False))
    param_list.append(('pix', pix, False))
    param_list.append(('xy', xy, False))
    param_list.append(('n_psfs', n_psfs, False))

    # Set independent variables
    indep_vars = {}

    # Get values from input parameters.Parameters instances
    initialParams.add_many(*param_list)

    # Create the lightcurve model
    specmodel = lmfit.Model(model)
    specmodel.independent_vars = indep_vars.keys()

    # Set the uncertainty
    if uncertainty is None:
#         uncertainty = np.ones_like(frame)
        uncertainty = np.ones_like(spectrum)

    # Fit model to the simulated data
#     result = specmodel.fit(frame, weights=1/uncertainty, params=initialParams, method=method, **indep_vars, **kwargs)
    result = specmodel.fit(spectrum, weights=1/uncertainty, params=initialParams, method=method, **indep_vars, **kwargs)
    
    # Plot the original spectrum
    fig = figure()
    fig.line(x, spectrum, legend='Input Spectrum', color='blue')
    
    # And the fit spectrum
    fit = calc_spectrum_soss(**result.best_values)
    fig.line(x, fit, legend='Best Fit Spectrum', color='green')
    show(fig)
    
    # Plot residuals
    fig = figure()
    fig.line(x, spectrum-fit, color='red')
    show(fig)
    
    # Plot the frame diff
    fit_frame = make_frame_soss(**result.best_values)
    fig = figure(tooltips=[('x', '$x'), ('y', '$y'), ("value", "@image")])
    data = dict(image=[frame])
    fig.image(image=[frame-fit_frame], x=0, y=0, dw=frame.shape[0], dh=frame.shape[1], palette="Inferno256")
    show(fig)
    
    # Return results
    return result



In [3]:
def psf(scale, pix=76, xy=50):
    """Make a psf of a given size and scale"""
    x, y = np.meshgrid(np.linspace(-1,1,pix), np.linspace(-1,1,pix))
    d = np.sqrt(xy*x*x+y*y)
    sigma, mu = 0.2, 0.0
    g = np.exp(-( (d-mu)**2 / ( 2.0 * sigma**2 ) ) )*scale
        
    return g

def make_frame(n_psfs, offset, pix, xy, plot=False, **scales):
    """Make a frame with N psfs with a pixel width and a pixel offset for each psf
    where each psf is scaled randomly"""
    # Set the values
    frame = np.zeros((76, 76+n_psfs))
    
    # Make the frame
    for n in range(n_psfs):
        frame[:, n:n+pix] += psf(scales['w{}'.format(n)], pix=pix)
        
    # Trim off excess
    frame = frame[:, 38:-38]
    
    if plot:
        # Plot the frame
        fig = figure(tooltips=[('x', '$x'), ('y', '$y'), ("value", "@image")])
        data = dict(image=[frame])
        fig.image(source=data, x=0, y=0, image='image', dw=frame.shape[0], dh=frame.shape[1], palette="Inferno256")
        show(fig)

    return frame

def calc_spectrum(n_psfs, offset, pix, xy, plot=False, **scales):
    """Calculate a spectrum by column sum"""
    frame = make_frame(n_psfs, offset, pix, xy, plot=plot, **scales)
    spectrum = np.sum(frame, axis=0)
    
    return spectrum

def lmfitter(frame, model, uncertainty=None, method='powell', verbose=True, **kwargs):
    """Use lmfit to find the scale of each psf

    Parameters
    ----------
    frame: sequence
        The frame data
    model: function
        The model to fit

    Returns
    -------
    lmfit.Model.fit.fit_report
        The results of the fit
    """
    # Make a spectrum
    spectrum = np.sum(frame, axis=0)
    x = np.arange(len(spectrum))
    
    # Initialize lmfit Params object
    initialParams = lmfit.Parameters()

    # Concatenate the lists of parameters
    param_list = [('w{}'.format(n), 1., True) for n in range(n_psfs)]
    param_list.append(('offset', offset, False))
    param_list.append(('pix', pix, False))
    param_list.append(('xy', xy, False))
    param_list.append(('n_psfs', n_psfs, False))

    # Set independent variables
    indep_vars = {}

    # Get values from input parameters.Parameters instances
    initialParams.add_many(*param_list)

    # Create the lightcurve model
    specmodel = lmfit.Model(model)
    specmodel.independent_vars = indep_vars.keys()

    # Set the uncertainty
    if uncertainty is None:
#         uncertainty = np.ones_like(frame)
        uncertainty = np.ones_like(spectrum)

    # Fit model to the simulated data
#     result = specmodel.fit(frame, weights=1/uncertainty, params=initialParams, method=method, **indep_vars, **kwargs)
    result = specmodel.fit(spectrum, weights=1/uncertainty, params=initialParams, method=method, **indep_vars, **kwargs)
    
    # Plot the original spectrum
    fig = figure()
    fig.line(x, spectrum, legend='Input Spectrum', color='blue')
    
    # And the fit spectrum
    fit = calc_spectrum(**result.best_values)
    fig.line(x, fit, legend='Best Fit Spectrum', color='green')
    show(fig)
    
    # Plot residuals
    fig = figure()
    fig.line(x, spectrum-fit, color='red')
    show(fig)
    
    # Plot the frame diff
    fit_frame = make_frame(**result.best_values)
    fig = figure(tooltips=[('x', '$x'), ('y', '$y'), ("value", "@image")])
    data = dict(image=[frame])
    fig.image(image=[frame-fit_frame], x=0, y=0, dw=frame.shape[0], dh=frame.shape[1], palette="Inferno256")
    show(fig)
    
    # Return results
    return result


## 2D Gaussians Scaled Randomly

In [5]:
# Set the parameters
n_psfs = 50
offset = 1
pix = 76
xy = 30
loc = 4000
scales = {'w{}'.format(n): v for n, v in enumerate(np.abs(np.random.normal(loc=loc, size=n_psfs, scale=loc/2)))}

# Make the frame
frame = make_frame(n_psfs=n_psfs, offset=offset, pix=pix, xy=xy, plot=True, **scales)

# Fit it
result = lmfitter(frame, calc_spectrum, method='leastsq')
# print(result.fit_report())

## 2D Gaussians Scaled to a Blackbody

In [6]:
# Make the blackbody
bb = models.BlackBody1D(temperature=2500*q.K)
wave = np.linspace(1, 4, 50)*q.um
flux = bb(wave).value*1e17
scales = {'w{}'.format(n): v for n, v in enumerate(flux)}

# Set the parameters
n_psfs = len(flux)
offset = 1
pix = 76
xy = 30

# Make the frame
frame = make_frame(n_psfs=n_psfs, offset=offset, pix=pix, xy=xy, plot=True, **scales)

# Fit it
result = lmfitter(frame, calc_spectrum, method='leastsq')
# print(result.fit_report())

## SOSS psf Scaled to a Blackbody

In [9]:
# Make the blackbody
bb = models.BlackBody1D(temperature=2500*q.K)
wave = np.linspace(0.9, 2.8, 50)*q.um
flux = bb(wave).value*1e18
scales = {'w{}'.format(n): v for n, v in enumerate(flux)}

# Set the parameters
n_psfs = len(flux)
offset = 1
pix = 76
xy = 30

# Get a SOSS psf
frame = make_frame_soss(n_psfs=n_psfs, offset=offset, pix=pix, xy=xy, plot=True, **scales)

# Fit it
result = lmfitter_soss(frame, calc_spectrum_soss, method='leastsq')
# print(result.fit_report())

## SOSS scaled to a spectrum (single psf)

In [8]:
# Get the spectrum
wave, flux, *_ = np.genfromtxt('/Users/jfilippazzo/Dropbox/BDNYC_spectra/SpeX/Prism/LHS132.txt', unpack=True)
print(flux.size)
scales = {'w{}'.format(n): v for n, v in enumerate(flux)}

# Set the parameters
n_psfs = len(flux)
offset = 1
pix = 76
xy = 30

# Make a SOSS trace
frame = make_frame_soss(n_psfs=n_psfs, offset=offset, pix=pix, xy=xy, plot=True, **scales)

# Fit it
result = lmfitter_soss(frame, calc_spectrum_soss, method='leastsq', truth=[wave, flux])

563


## SOSS scaled to a spectrum (wavelength dependent psfs)

In [10]:
# Get the spectrum
wave, flux, *_ = np.genfromtxt('/Users/jfilippazzo/Dropbox/BDNYC_spectra/SpeX/Prism/LHS132.txt', unpack=True)

# Interpolate to GR700XD

print(flux.size)
scales = {'w{}'.format(n): v for n, v in enumerate(flux)}

# Set the parameters
n_psfs = len(flux)
offset = 1
pix = 76
xy = 30

# Make a SOSS trace
frame = make_frame_soss(n_psfs=n_psfs, offset=offset, pix=pix, xy=xy, plot=True, **scales)

# Fit it
result = lmfitter_soss(frame, calc_spectrum_soss, method='leastsq', truth=[wave, flux])

563


## Actual SOSS simulation (no noise)

## Actual SOSS simulation (with noise)
This requires a working version of the `run_pipeline` method